In [39]:
import time
import pandas as pd
import numpy as np
from numpy import array
import torch
import torch.nn as nn
from sklearn.model_selection import KFold

In [40]:
save_path = "/Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/"
model_name = "fourier_two_features_2"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 60

In [41]:
#### define functions ####

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, out_dim):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, out_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [42]:
def display_fits(fits):
    print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 2])}",
          f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 2])}",
          f"\nAverage correlation on random: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 1])}",
          f"\nAverage MSE on random: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 1])}",
          f"\nAverage correlation on ChrV: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 3])}",
          f"\nAverage MSE on ChrV: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 3])}",
          f"\nAverage correlation on CN: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 0])}",
          f"\nAverage MSE on CN: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 0])}")

In [43]:
# Construct sine and cosine matrices for Fourier features:
cos_matrix, sin_matrix = [[] for x in range(49)], [[] for x in range(49)]
for n in range(49):
    for k in range(49):
        cos_matrix[n].append(np.cos(2*np.pi*(n+1)*k/49))
        sin_matrix[n].append(np.sin(2*np.pi*(n+1)*k/49))

In [51]:
def dnaTwoFourier(sequence):
    seq_array = array(list(sequence))
    code = {"AA": [1], "AT": [1], "TA": [1], "TT": [1],
            "CA": [0], "CC": [0], "CG": [0], "CT": [0],
            "GA": [0], "GC": [0], "GG": [0], "GT": [0],
            "AC": [0], "TC": [0], "TG": [0], "AG": [0],
            "aa": [1], "at": [1], "ta": [1], "tt": [1],
            "ca": [0], "cc": [0], "cg": [0], "ct": [0],
            "ga": [0], "gc": [0], "gg": [0], "gt": [0],
            "ac": [0], "tc": [0], "tg": [0], "ag": [0],}
    AT_encoded_seq = []
    for i in range(len(seq_array)-1):
        AT_encoded_seq.append(code["".join((seq_array[i], seq_array[i+1]))][0])
    two_fourier_cos = np.matmul(array(AT_encoded_seq), array(cos_matrix).transpose())
    two_fourier_sin = np.matmul(array(AT_encoded_seq), array(sin_matrix).transpose())
    return list(np.concatenate((two_fourier_cos, two_fourier_sin)))

In [ ]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], inv_mat))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [53]:
#### preparing data ####

data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle1.txt",delimiter = ",")
X1_all = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1_all.append(dnaTwoFourier(sequence_nt))
X1_all = np.float32(array(X1))
# X1 = X1.reshape((X1.shape[0],50,4,1))
### COMPUTE EIGENVECTORS:

# X1_reverse = np.flip(X1,[1,2])
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)
Y1 = np.float32(find_c0new(data_cerevisiae_nucle).astype(float))


In [54]:
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle3.txt",delimiter = ",")
X3_all = []
for sequence_nt in data_random_library["Sequence"]:
    X3_all.append(dnaTwoFourier(sequence_nt))
X3_all = np.float32(array(X3_all))
# X3 = X3.reshape((X3.shape[0],50,2,1))
# X3_reverse = np.flip(X3,[1,2])
# Y3 = data_random_library["C0"].values.astype(float)
Y3 = np.float32(find_c0new(data_random_library).astype(float))

In [55]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle5.txt",delimiter = ",")
X5_all = []
for sequence_nt in data_tiling["Sequence"]:
    X5_all.append(dnaTwoFourier(sequence_nt))
X5_all = np.float32(array(X5_all))
# X5 = X5.reshape((X5.shape[0],50,4,1))
# X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = np.float32(find_c0new(data_tiling).astype(float))

In [60]:
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle6.txt",delimiter = ",")
X6_all = []
for sequence_nt in data_chr5["Sequence"]:
    X6_all.append(dnaTwoFourier(sequence_nt))
X6_all = np.float32(array(X6_all))
# X6 = X6.reshape((X6.shape[0],50,4,1))
# X6_reverse = np.flip(X6,[1,2])
# Y6 = data_chr5["C0"].values.astype(float)
Y6 = np.float32(find_c0new(data_chr5).astype(float))

In [64]:
which_feature = 4
X1 = X1_all[:,(which_feature, which_feature + 49)]
X3 = X3_all[:,(which_feature, which_feature + 49)]
X5 = X5_all[:,(which_feature, which_feature + 49)]
X6 = X6_all[:,(which_feature, which_feature + 49)]

In [65]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
n = Y5.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Y5):
    training_X = np.float32(X5[train_index])
    validation_X = np.float32(X5[val_index])
    training_Y = np.float32(Y5[train_index])
    validation_Y = np.float32(Y5[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=2, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 2)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(X1)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X3)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X6)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 0.4561
Epoch [10/60], Loss: 0.1309
Epoch [15/60], Loss: 0.1181
Epoch [20/60], Loss: 0.1168
Epoch [25/60], Loss: 0.1163
Epoch [30/60], Loss: 0.1160
Epoch [35/60], Loss: 0.1157
Epoch [40/60], Loss: 0.1155
Epoch [45/60], Loss: 0.1153
Epoch [50/60], Loss: 0.1152
Epoch [55/60], Loss: 0.1151
Epoch [60/60], Loss: 0.1152
Epoch [5/60], Loss: 0.1246
Epoch [10/60], Loss: 0.1204
Epoch [15/60], Loss: 0.1184
Epoch [20/60], Loss: 0.1171
Epoch [25/60], Loss: 0.1163
Epoch [30/60], Loss: 0.1157
Epoch [35/60], Loss: 0.1154
Epoch [40/60], Loss: 0.1151
Epoch [45/60], Loss: 0.1150
Epoch [50/60], Loss: 0.1148
Epoch [55/60], Loss: 0.1148
Epoch [60/60], Loss: 0.1147
Epoch [5/60], Loss: 0.1283
Epoch [10/60], Loss: 0.1228
Epoch [15/60], Loss: 0.1209
Epoch [20/60], Loss: 0.1197
Epoch [25/60], Loss: 0.1189
Epoch [30/60], Loss: 0.1182
Epoch [35/60], Loss: 0.1177
Epoch [40/60], Loss: 0.1173
Epoch [45/60], Loss: 0.1170
Epoch [50/60], Loss: 0.1167
Epoch [55/60], Loss: 0.1164
Epoch [60/60], Loss: 0.

In [66]:
display_fits(fits)

Average correlation on tiling: 0.5255317544707367 
Average MSE on tiling: 0.11599666774272918 
Average correlation on random: 0.48641894369688166 
Average MSE on random: 0.09559891000390053 
Average correlation on ChrV: 0.41171093958057936 
Average MSE on ChrV: 0.2137158066034317 
Average correlation on CN: 0.5200955398007451 
Average MSE on CN: 0.15908850878477096
